# Workshop_09
## Immagine di esempio:

Partendo dalla seguente immagine di esempio, il seguente workshop è in grado di creare un "tetto con terrazzo" a partire da qualsiasi poligono p assegnato.

<img src='./Immagini/esempio1.PNG'>

## Codice:
Vengono usate più funzioni:
* Una funzione primaria *ggpl_create_roof()* che assembla il tetto costruito tramite l'utilizzo di alcune funzioni ausiliarie:
    1. Una funzione *create_segment_faces(V)* che data una lista di vertici restituisce la lista di segmenti del poligono
    2. Una funzione *get_plane_pitch(A,B,z)* che costruisce una falda piana sul piano XY della lungheza desiderata
    3. Una funzione *get_sloping_pitch(A,B,z,rad)* che data la falda piana la inclina di rad radianti
    4. Le funzioni *line_from_two_points(A,B)* e *line_line_intersection(line1,line2)* che calcolano rispettivamente l'equazione della retta passante per due punti e le coordinate del punto d'intersezione fra due rette.
    5. Una funzione *create_pitch(V1,V2)* che date due liste di vertici rappresentanti il perimetro inferiore e superiore del tetto, restituiscono la struttura delle falde
    
L'idea è quella di creare per prima cosa delle falde inclinate nel corretto verso di inclinazione partendo da ogni segmento del poligono. Prendere poi per ogni falda i punti ad altezza h!=0, calcolare la retta per due punti ed intersecarla con quella successiva in modo da ottenere iterativamente la lista dei vertici del perimetro superiore del tetto. Con le due liste di vertici perimetrali posso creare ogni falda utilizzando la funzione MKPOL. 

In [3]:
from larlib import *
from numpy import *

def ggpl_create_roof(V,F,z,rad):
    
    #FUNZIONI AUSILIARIE ------------------------------------------------------
    
    def create_segments_faces(V):
        #ogni segmento è identificato da una faccia bidimensionale
        #esempio [[0,1],[1,2],...]
        lines = []
        for i in range(len(V)-1):
            lines.append([i+1,i+2])
        lines.append([len(V),1])
        return lines
    
    """retta per 2 punti"""
    def line_from_two_points(A,B):
        x1,y1,z1 = A
        x2,y2,z2 = B

        a = y1 - y2
        b = x2 - x1
        c = x1*y2 - x2*y1

        return [a,b,-c]
    
    """tramite Cramer calcolo il punto x,y d'intersezione fra due rette"""
    def line_line_intersection(r1, r2):
        D  = r1[0] * r2[1] - r1[1] * r2[0]
        Dx = r1[2] * r2[1] - r1[1] * r2[2]
        Dy = r1[0] * r2[2] - r1[2] * r2[0]

        if D!=0:
            x = Dx/D
            y = Dy/D
            return [x,y]
        else:
            return False
    
    def get_plane_pitch(A,B,z):
        x1,y1,z1 = A
        x2,y2,z2 = B

        angle = ATAN2([(y1-y2),(x2-x1)])

        A1 = [x1+COS(angle)*z,y1+SIN(angle)*z,z1]
        B2 = [x2+COS(angle)*z,y2+SIN(angle)*z,z2]

        planePitch = STRUCT([MKPOL([[A,B,A1,B2],[[1,2,3,4]],None])])
        return planePitch
    
    def get_sloping_pitch(A,B,z,rad):
        x1,y1,z1 = A
        x2,y2,z2 = B

        planePitch = get_plane_pitch(A,B,z) 
        angle = ATAN2([(x2-x1),(y1-y2)])
        #print angle*180/PI

        originPitch = T([1,2])([-A[0], -A[1]])(planePitch)
        originPitch = R([1,2])(angle)(originPitch)
        rotation = R([2,3])(rad)(originPitch)
        originPitch = R([1,2])(-angle)(rotation)

        slopingPitch = T([1,2])([A[0],A[1]])(originPitch)
        return slopingPitch
    
    def create_pitch(V, interPoints):
        VTemp = V + [V[0]] + interPoints + [interPoints[0]]
        pitches = []
        for i in range(len(V)):
            points = [VTemp[i],VTemp[i+1],VTemp[i+1+len(V)],VTemp[i+2+len(V)]]
            pitch = STRUCT(MKPOLS([points,[[0,1,2,3]],None]))
            pitches.append(pitch)
        return TEXTURE(['./immagini/roof_texture.jpg',True, False, 1, 1, PI/2., 5,5])(STRUCT(pitches))

    #END FUNZIONI AUSILIARIE---------------------------------------------------
    
    segments = create_segments_faces(V)
    pitches = []
    for i in segments:
        A = V[i[0]-1]
        B = V[i[1]-1]
        slopingPitch = get_sloping_pitch(A,B,z,rad)
        pitches.append(slopingPitch)

    #VIEW(STRUCT(pitches))

    lines = []
    for i in range(len(pitches)):
        #troviamo i punti di ogni falda: quelli con z != 0 
        #son quelli per cui vado a calcolare la retta per due punti
        pol1 = UKPOL(pitches[i])[0]
        points = []
        for j in range(len(pol1)):
            if (round(pol1[j][2],3) != 0.0) :
                h = pol1[j][2]
                points.append([round(k,3) for k in pol1[j]])
        lines.append(line_from_two_points(points[0],points[1]))

    interPoint = line_line_intersection(lines[len(lines)-1], lines[0]) + [h]
    interPoints = [[round(k,3) for k in interPoint]]
    for i in range(len(lines)-1):
        interPoint = line_line_intersection(lines[i], lines[i+1]) + [h]
        interPoints.append([round(k,3) for k in interPoint])

    platform = TEXTURE(['./immagini/roof_texture.jpg',True, False, 1, 1, PI/2., 5,5])(STRUCT([MKPOL([interPoints,F,1])]))
    #VIEW(platform)
    
    #nel caso in cui non avessi a disposizione le faces del poligono:
    #
    #polyline = []
    #V2d = []
    #for point in V:
    #   V2d.append([point[0],point[1]])
    #V2d.append([V[0][0],V[0][1]])
    #platform = SOLIDIFY(STRUCT([POLYLINE(V2d)]))
    
    pitches = create_pitch(V,interPoints)
    #VIEW(pitches)

    return STRUCT([platform, pitches])

## Esempi

In [5]:
V= [[0,0,0],[4,0,0],[4,4,0],[8,2,0],[8,7,0],[0,7,0]]
F= [[1,2,3,6],[3,4,5,6]]
z = .8
rad = PI/4

VIEW(ggpl_create_roof(V,F,z,rad))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001B9D5C00> >

<img src='./Immagini/test1.PNG'>

In [7]:
V= [[0,0,0],[4,0,0],[4,4,0],[8,4,0],[8,0,0],[12,0,0],[12,8,0],[0,8,0]]
F= [[1,2,3,8],[3,4,7,8],[4,5,6,7]]
z = 1.5
rad = PI/4

VIEW(ggpl_create_roof(V,F,z,rad))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x00000000049C7300> >

<img src='./Immagini/test2.PNG'>